In [1]:
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
# Ensure you have NLTK stopwords downloaded
## Stop Words
nltk.download('stopwords')
# additional_stopwords=['celery', 'airflow', 'snakemake', 'nextflow', 'knime', 'galaxy', 'oozie', 'drake','pydrake' 'luigi', 'dask', 'nifi']
additional_stopwords=['perf', 'pr', 'issue', 'bump', 'autobump', 'choremain']
stop_words = set(stopwords.words('english'))
stop_words.update(additional_stopwords)


nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/kha060/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kha060/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kha060/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Function to remove code snippets from the body
# Function to remove HTTPS links from text
def remove_https_links(text):
    # Ensure the input is a string
    if isinstance(text, str):
        # Regular expression pattern to match HTTPS links
        https_pattern = r'https://\S+'
        # Remove all matches from the text
        return re.sub(https_pattern, '', text).strip()
    else:
        # If not a string, return as is (or handle as needed)
        return text
def remove_code_snippets(body):
    if not body:
        return body
    # Remove inline code (e.g., `code`)
    body = re.sub(r'`[^`]*`', '', body)
    # Remove block code (e.g., ```code```)
    body = re.sub(r'```[\s\S]*?```', '', body)
    return body

# Function to remove non-alphabetical characters
def keep_only_alphabetical(body):
    if not body:
        return body
    # Replace non-alphabetical characters (except spaces) with an empty string
    return re.sub(r'[^a-zA-Z\s]', '', body)

# Function to remove punctuation and newline characters from a text
def clean_text(text):
    if text is None:
        return text
    # Create a translation table for removing punctuation
    translator = str.maketrans('', '', string.punctuation)
    # Remove punctuation
    text = text.translate(translator)
    # Remove newline characters
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    return text

## Remove stopwords

def remove_stop_words(text):
    if text is None:
        return text
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    if text is None:
        return text
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Airflow Issues

In [4]:
df_airflow = pd.read_json('apache_airflow_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_airflow = df_airflow[columns_to_keep]
df_filtered_airflow

,id,title,body,created_at,closed_at,state
0,2428317348,Inconsistent data_interval_start/data_interval...,### Apache Airflow version\n\nOther Airflow 2 ...,2024-07-24 19:25:54+00:00,NaT,open
1,2428215155,Setting the language of textarea field through...,### Description\r\n\r\nHi\r\nWhile working on ...,2024-07-24 18:24:08+00:00,NaT,open
2,2427966039,Add option to hide Datasets in Graph View,### Description\n\nIf we have a lot of dataset...,2024-07-24 16:03:06+00:00,NaT,open
3,2427473325,"GKEStartPodOperator Error pod ""No agent availa...",### Apache Airflow Provider(s)\n\ngoogle\n\n##...,2024-07-24 12:34:23+00:00,NaT,open
4,2427028176,Airflow Backend Database RDS PostgreSQL high C...,### Apache Airflow version\n\nOther Airflow 2 ...,2024-07-24 09:19:06+00:00,2024-07-24 12:27:24+00:00,closed
...,...,...,...,...,...,...
9326,85640444,Does airflow handle memory management ?,I didn't any ui pictures about that.\n,2015-06-05 21:54:47+00:00,2015-06-11 20:10:27+00:00,closed
9327,85614853,Scheduling frequency,Is there a way to schedule jobs/dags on specif...,2015-06-05 19:27:53+00:00,2015-06-11 03:27:32+00:00,closed
9328,85613531,Can you have a DAG dependent on another DAG?,I see `airflow.operators.ExternalTaskSensor` s...,2015-06-05 19:21:58+00:00,2015-06-11 20:03:20+00:00,closed
9329,85557204,setup.py download URL,While installing I noticed the `url` and `down...,2015-06-05 15:18:52+00:00,2015-06-05 17:24:57+00:00,closed


In [5]:
# Apply the function to the 'body' and title column For Airflow
df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(remove_https_links)
df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(remove_https_links)

df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(remove_code_snippets)
df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(remove_code_snippets)

df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(keep_only_alphabetical)
df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(keep_only_alphabetical)

df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(clean_text)
df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(clean_text)

df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(remove_stop_words)
df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(remove_stop_words)


df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(lemmatize_text)
df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(lemmatize_text)

df_filtered_airflow.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_Airflow_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/593989495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_airflow['body'] = df_filtered_airflow['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/593989495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_airflow['title'] = df_filtered_airflow['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/593989495.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [6]:
# df = pd.read_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_Airflow_Issues.json',  orient='records', lines=True)
# df

# Galaxy Issues

In [7]:
df_galaxy = pd.read_json('galaxyproject_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_galaxy = df_galaxy[columns_to_keep]

# Apply the function to the 'body' and title column For galaxy
df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(remove_https_links)
df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(remove_https_links)

df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(remove_code_snippets)
df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(remove_code_snippets)

df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(keep_only_alphabetical)
df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(keep_only_alphabetical)

df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(clean_text)
df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(clean_text)

df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(remove_stop_words)
df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(remove_stop_words)


df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(lemmatize_text)
df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(lemmatize_text)

df_filtered_galaxy.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_galaxy_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/1365757952.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_galaxy['body'] = df_filtered_galaxy['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/1365757952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_galaxy['title'] = df_filtered_galaxy['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/1365757952.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

# SnakeMake

In [8]:
df_snakemake = pd.read_json('snakemake_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_snakemake = df_snakemake[columns_to_keep]

# Apply the function to the 'body' and title column For snakemake
df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(remove_https_links)
df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(remove_https_links)

df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(remove_code_snippets)
df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(remove_code_snippets)

df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(keep_only_alphabetical)
df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(keep_only_alphabetical)

df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(clean_text)
df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(clean_text)

df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(remove_stop_words)
df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(remove_stop_words)


df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(lemmatize_text)
df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(lemmatize_text)

df_filtered_snakemake.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_snakemake_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/660304976.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_snakemake['body'] = df_filtered_snakemake['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/660304976.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_snakemake['title'] = df_filtered_snakemake['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/660304976.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

# NextFlow

In [9]:
df_nextflow = pd.read_json('nextflow-io_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_nextflow = df_nextflow[columns_to_keep]

# Apply the function to the 'body' and title column For nextflow
df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(remove_https_links)
df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(remove_https_links)

df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(remove_code_snippets)
df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(remove_code_snippets)

df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(keep_only_alphabetical)
df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(keep_only_alphabetical)

df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(clean_text)
df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(clean_text)

df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(remove_stop_words)
df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(remove_stop_words)


df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(lemmatize_text)
df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(lemmatize_text)

df_filtered_nextflow.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_nextflow_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/3269953269.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_nextflow['body'] = df_filtered_nextflow['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/3269953269.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_nextflow['title'] = df_filtered_nextflow['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/3269953269.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

# Celery

In [10]:
df_celery = pd.read_json('celery_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_celery = df_celery[columns_to_keep]

# Apply the function to the 'body' and title column For celery
df_filtered_celery['body'] = df_filtered_celery['body'].apply(remove_https_links)
df_filtered_celery['title'] = df_filtered_celery['title'].apply(remove_https_links)

df_filtered_celery['body'] = df_filtered_celery['body'].apply(remove_code_snippets)
df_filtered_celery['title'] = df_filtered_celery['title'].apply(remove_code_snippets)

df_filtered_celery['title'] = df_filtered_celery['title'].apply(keep_only_alphabetical)
df_filtered_celery['body'] = df_filtered_celery['body'].apply(keep_only_alphabetical)

df_filtered_celery['body'] = df_filtered_celery['body'].apply(clean_text)
df_filtered_celery['title'] = df_filtered_celery['title'].apply(clean_text)

df_filtered_celery['title'] = df_filtered_celery['title'].apply(remove_stop_words)
df_filtered_celery['body'] = df_filtered_celery['body'].apply(remove_stop_words)


df_filtered_celery['title'] = df_filtered_celery['title'].apply(lemmatize_text)
df_filtered_celery['body'] = df_filtered_celery['body'].apply(lemmatize_text)

df_filtered_celery.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_celery_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/1761627183.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_celery['body'] = df_filtered_celery['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/1761627183.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_celery['title'] = df_filtered_celery['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/1761627183.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

# KNIME

In [11]:
df_knime = pd.read_json('knime_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_knime = df_knime[columns_to_keep]

# Apply the function to the 'body' and title column For knime
df_filtered_knime['body'] = df_filtered_knime['body'].apply(remove_https_links)
df_filtered_knime['title'] = df_filtered_knime['title'].apply(remove_https_links)

df_filtered_knime['body'] = df_filtered_knime['body'].apply(remove_code_snippets)
df_filtered_knime['title'] = df_filtered_knime['title'].apply(remove_code_snippets)

df_filtered_knime['title'] = df_filtered_knime['title'].apply(keep_only_alphabetical)
df_filtered_knime['body'] = df_filtered_knime['body'].apply(keep_only_alphabetical)

df_filtered_knime['body'] = df_filtered_knime['body'].apply(clean_text)
df_filtered_knime['title'] = df_filtered_knime['title'].apply(clean_text)

df_filtered_knime['title'] = df_filtered_knime['title'].apply(remove_stop_words)
df_filtered_knime['body'] = df_filtered_knime['body'].apply(remove_stop_words)


df_filtered_knime['title'] = df_filtered_knime['title'].apply(lemmatize_text)
df_filtered_knime['body'] = df_filtered_knime['body'].apply(lemmatize_text)

df_filtered_knime.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_knime_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/3582190390.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_knime['body'] = df_filtered_knime['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/3582190390.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_knime['title'] = df_filtered_knime['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/3582190390.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# Drake

In [12]:
df_drake = pd.read_json('ropensci_drake.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_drake = df_drake[columns_to_keep]

# Apply the function to the 'body' and title column For drake
df_filtered_drake['body'] = df_filtered_drake['body'].apply(remove_https_links)
df_filtered_drake['title'] = df_filtered_drake['title'].apply(remove_https_links)

df_filtered_drake['body'] = df_filtered_drake['body'].apply(remove_code_snippets)
df_filtered_drake['title'] = df_filtered_drake['title'].apply(remove_code_snippets)

df_filtered_drake['title'] = df_filtered_drake['title'].apply(keep_only_alphabetical)
df_filtered_drake['body'] = df_filtered_drake['body'].apply(keep_only_alphabetical)

df_filtered_drake['body'] = df_filtered_drake['body'].apply(clean_text)
df_filtered_drake['title'] = df_filtered_drake['title'].apply(clean_text)

df_filtered_drake['title'] = df_filtered_drake['title'].apply(remove_stop_words)
df_filtered_drake['body'] = df_filtered_drake['body'].apply(remove_stop_words)


df_filtered_drake['title'] = df_filtered_drake['title'].apply(lemmatize_text)
df_filtered_drake['body'] = df_filtered_drake['body'].apply(lemmatize_text)

df_filtered_drake.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_drake_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/2291012327.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_drake['body'] = df_filtered_drake['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/2291012327.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_drake['title'] = df_filtered_drake['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/2291012327.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# Luigi

In [13]:
df_luigi = pd.read_json('spotify_luigi.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_luigi = df_luigi[columns_to_keep]

# Apply the function to the 'body' and title column For luigi

df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(remove_https_links)
df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(remove_https_links)

df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(remove_code_snippets)
df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(remove_code_snippets)

df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(keep_only_alphabetical)
df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(keep_only_alphabetical)

df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(clean_text)
df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(clean_text)

df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(remove_stop_words)
df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(remove_stop_words)


df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(lemmatize_text)
df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(lemmatize_text)

df_filtered_luigi.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_luigi_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/4113698793.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_luigi['body'] = df_filtered_luigi['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/4113698793.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_luigi['title'] = df_filtered_luigi['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/4113698793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# Dask

In [14]:
df_dask = pd.read_json('dask_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_dask = df_dask[columns_to_keep]

# Apply the function to the 'body' and title column For dask

df_filtered_dask['body'] = df_filtered_dask['body'].apply(remove_https_links)
df_filtered_dask['title'] = df_filtered_dask['title'].apply(remove_https_links)

df_filtered_dask['body'] = df_filtered_dask['body'].apply(remove_code_snippets)
df_filtered_dask['title'] = df_filtered_dask['title'].apply(remove_code_snippets)

df_filtered_dask['title'] = df_filtered_dask['title'].apply(keep_only_alphabetical)
df_filtered_dask['body'] = df_filtered_dask['body'].apply(keep_only_alphabetical)

df_filtered_dask['body'] = df_filtered_dask['body'].apply(clean_text)
df_filtered_dask['title'] = df_filtered_dask['title'].apply(clean_text)

df_filtered_dask['title'] = df_filtered_dask['title'].apply(remove_stop_words)
df_filtered_dask['body'] = df_filtered_dask['body'].apply(remove_stop_words)


df_filtered_dask['title'] = df_filtered_dask['title'].apply(lemmatize_text)
df_filtered_dask['body'] = df_filtered_dask['body'].apply(lemmatize_text)

df_filtered_dask.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_dask_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/1664835500.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_dask['body'] = df_filtered_dask['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/1664835500.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_dask['title'] = df_filtered_dask['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/1664835500.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [15]:
df_nifi = pd.read_json('apache_nifi_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_nifi = df_nifi[columns_to_keep]

# Apply the function to the 'body' and title column For nifi

df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(remove_https_links)
df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(remove_https_links)

df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(remove_code_snippets)
df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(remove_code_snippets)

df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(keep_only_alphabetical)
df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(keep_only_alphabetical)

df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(clean_text)
df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(clean_text)

df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(remove_stop_words)
df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(remove_stop_words)


df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(lemmatize_text)
df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(lemmatize_text)

df_filtered_nifi.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_nifi_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/2349645808.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_nifi['body'] = df_filtered_nifi['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/2349645808.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_nifi['title'] = df_filtered_nifi['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/2349645808.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [16]:
df_oozie = pd.read_json('oozie_issues.json')
# Select specific columns
columns_to_keep = ['id', 'title', 'body', 'created_at', 'closed_at', 'state']
df_filtered_oozie = df_oozie[columns_to_keep]

# Apply the function to the 'body' and title column For oozie

df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(remove_https_links)
df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(remove_https_links)

df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(remove_code_snippets)
df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(remove_code_snippets)

df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(keep_only_alphabetical)
df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(keep_only_alphabetical)

df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(clean_text)
df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(clean_text)

df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(remove_stop_words)
df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(remove_stop_words)


df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(lemmatize_text)
df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(lemmatize_text)

df_filtered_oozie.to_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_oozie_Issues.json', orient='records', lines=True)

/tmp/ipykernel_2248030/708320991.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_oozie['body'] = df_filtered_oozie['body'].apply(remove_https_links)
/tmp/ipykernel_2248030/708320991.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_oozie['title'] = df_filtered_oozie['title'].apply(remove_https_links)
/tmp/ipykernel_2248030/708320991.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [17]:
df = pd.read_json('../../Replication Package/Data/GitHub Issues and Pull Requests/Preprocessed_oozie_Issues.json',  orient='records', lines=True)
df

,id,title,body,created_at,closed_at,state
0,194929338,oozie run configuration xml replace local conf...,call jobRun method special parameter run coord...,2016-12-12 09:44:22,NaT,open
1,194891349,oozie get wrong job state,hi oozie team got promble use oozie lanuch spa...,2016-12-12 05:01:52,NaT,open
2,173478698,Using EL function jobproperties,Hi guy wrote custom EL function load ENV varia...,2016-08-26 15:21:50,NaT,open
3,104905384,oozie coordinator inputevent work,hi three coordinator B C coordinator B C depen...,2015-09-04 14:49:41,NaT,open
4,5064575,Unable ssh remote machine oozie,oozie vesion cdhu Trying execute shell command...,2012-06-14 08:28:07,NaT,open
...,...,...,...,...,...,...
379,308435,buildsetupsetupmavensh us bashisms reference b...,Script defines function therefore fails system...,2010-09-08 23:12:28,2010-09-14 22:16:49,closed
380,260087,Oozie console reference RowExpanderjs file inc...,indexhtml file launch oozie console reference ...,2010-07-26 19:47:20,2010-09-29 00:11:37,closed
381,255786,XTestCasegetTestUser method return hardcoded u...,One workaround would run test test user enviro...,2010-07-21 20:20:33,2010-09-29 00:11:37,closed
382,250646,binmkdistrosh DskipTests dmysql uroot llocalho...,run get mvn clean install DbuildtimeGMT Dsvnre...,2010-07-16 13:50:02,2010-09-29 04:42:21,closed
